In [2]:
import pandas as pd
import numpy as np

In [10]:
df = pd.read_excel('NutriScore_Dynamic_Dataset.xlsx', sheet_name='Sheet1')
df.head()

,Nom_Produit,Marque,Categorie,Energie_kJ,Sucres_g,Graisses_Sat_g,Sel_g,Proteines_g,Fibres_g,Fruits_Legumes_Pct,NutriScore_Score,NutriScore_Lettre,Points_N,Points_P,GreenScore_Lettre,GreenScore_Score
0,Concentré de tomates Aicha,Aicha,vegetables,132.0,2.43,0.0,0.270,1.24,0.811,0.0,1,B,1,0,C,59
1,Aicha Tomato Paste,Aicha,vegetables,496.0,9.00,0.0,0.500,4.60,3.000,0.0,3,C,4,1,C,46
2,Raw kimchi,Vadasz,vegetables,63.0,0.00,0.1,2.065,1.10,1.800,0.0,10,C,10,0,A,88
3,Beurre de cacahuètes,Maribel,vegetables,2860.0,3.90,12.0,0.030,30.40,9.200,0.0,14,D,19,5,D,34
4,Carottes rapees citron dnp 320g,Bonduelle,vegetables,301.0,4.50,0.3,0.710,1.00,3.000,86.0,-1,A,4,5,A,79


In [11]:
df['Nom_Produit'] = df['Nom_Produit'].str.replace(r'[^\w\s]', '', regex=True)
df.head()

,Nom_Produit,Marque,Categorie,Energie_kJ,Sucres_g,Graisses_Sat_g,Sel_g,Proteines_g,Fibres_g,Fruits_Legumes_Pct,NutriScore_Score,NutriScore_Lettre,Points_N,Points_P,GreenScore_Lettre,GreenScore_Score
0,Concentré de tomates Aicha,Aicha,vegetables,132.0,2.43,0.0,0.270,1.24,0.811,0.0,1,B,1,0,C,59
1,Aicha Tomato Paste,Aicha,vegetables,496.0,9.00,0.0,0.500,4.60,3.000,0.0,3,C,4,1,C,46
2,Raw kimchi,Vadasz,vegetables,63.0,0.00,0.1,2.065,1.10,1.800,0.0,10,C,10,0,A,88
3,Beurre de cacahuètes,Maribel,vegetables,2860.0,3.90,12.0,0.030,30.40,9.200,0.0,14,D,19,5,D,34
4,Carottes rapees citron dnp 320g,Bonduelle,vegetables,301.0,4.50,0.3,0.710,1.00,3.000,86.0,-1,A,4,5,A,79


In [ ]:
df.to_csv('NutriScore_Dynamic_Dataset_Cleaned.csv', index=False)